# Лабораторная работа №5: Представление графа. Обход графа

Лабораторная работа состоит из 6 задач. Для каждой задачи вы должны дать: \
1) код, где первый комментарий - асимптотическая сложность алгоритма (по времени и по памяти!) \

2) скрин с окнутой задачей из тестирующей системы\

3) ВАШЕ (не гпт) описание работы (так называемая, интуиция / идея решения).

Задача засчитывается, если три пункта выполнены честно.

Если вдруг, обнаружится плагиат / гпт решения - аннулируется задача без возможности апелляции.

# Приступим


## Задача: удаление клеток ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=50&id_problem=1013)) [1 балл]

Из прямоугольного листа клетчатой бумаги размером N строк и M столбцов удалили некоторые клетки. Если клетка не была удалена, она обозначается символом `#`, а если удалена – символом `.`. Остальные клетки, которые имеют общую сторону, считаются соединёнными и образуют один кусок бумаги. Необходимо определить, на сколько частей распадётся оставшаяся часть листа после удаления клеток.

---

## Входные данные

- В первой строке входного файла `INPUT.TXT` содержатся два целых числа N и M (1 ≤ N, M ≤ 100) – количество строк и столбцов листа.
- Далее следуют N строк, каждая из которых содержит M символов. Символ `#` означает, что клетка осталась, а символ `.` – что клетка была удалена.

---

## Выходные данные

Выведите одно число – количество отдельных кусков, на которые распадётся оставшаяся часть листа. Две клетки считаются принадлежащими одному куску, если они имеют общую сторону.

---

## Пример

**Входные данные:**
```
4 8
#.##.#.#
......##
#.###.##
##.##.##
```

**Выходные данные:**
```
6
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
# -*- coding: utf-8 -*-
import sys
from typing import List, Tuple

# Увеличиваем лимит глубины рекурсии для предотвращения RecursionError
# на больших или сложных сетках в пределах N, M <= 100.
# Максимальная глубина может быть N * M = 100 * 100 = 10000.
# Устанавливаем с запасом.
try:
    # Устанавливаем лимит рекурсии (например, 10500)
    sys.setrecursionlimit(10500)
except Exception as e:
    # В некоторых средах изменение лимита может быть ограничено
    # Выводим предупреждение в stderr, чтобы не мешать выводу в OUTPUT.TXT
    print(f"Предупреждение: Не удалось установить лимит рекурсии: {e}. "
          "Возможны проблемы на больших входах.", file=sys.stderr)

def solve() -> None:
    """
    Основная функция для решения задачи поиска связных компонент.
    Использует поиск в глубину (DFS).
    """
    # --- 1. Чтение входных данных ---
    try:
        with open('INPUT.TXT', 'r', encoding='utf-8') as f:
            # Читаем размеры сетки N (строки) и M (столбцы)
            line1 = f.readline()
            if not line1:
                print("Ошибка: Файл INPUT.TXT пуст или первая строка отсутствует.", file=sys.stderr)
                # В случае ошибки создаем пустой OUTPUT.TXT или с 0, чтобы система тестирования не зависла
                try:
                    open('OUTPUT.TXT', 'w').close()
                except:
                    pass
                return
            try:
                n, m = map(int, line1.split())
            except ValueError:
                print("Ошибка: Некорректный формат первой строки в INPUT.TXT (ожидаются два целых числа, разделенных пробелом).", file=sys.stderr)
                try:
                    open('OUTPUT.TXT', 'w').close()
                except:
                    pass
                return

            # Читаем саму сетку
            grid = [f.readline().strip() for _ in range(n)]

    except FileNotFoundError:
        print("Ошибка: Файл INPUT.TXT не найден.", file=sys.stderr)
        try:
            open('OUTPUT.TXT', 'w').close()
        except:
            pass
        return
    except Exception as e:
        print(f"Ошибка при чтении файла INPUT.TXT: {e}", file=sys.stderr)
        try:
            open('OUTPUT.TXT', 'w').close()
        except:
            pass
        return

    # --- Валидация входных данных ---
    validation_error = False
    if not (1 <= n <= 100 and 1 <= m <= 100):
        print(f"Ошибка: Размеры сетки N={n}, M={m} выходят за пределы ограничений [1, 100].", file=sys.stderr)
        validation_error = True
    elif len(grid) != n:
        print(f"Ошибка: Ожидалось {n} строк сетки, но прочитано {len(grid)}.", file=sys.stderr)
        validation_error = True
    else:
        for i, row in enumerate(grid):
            if len(row) != m:
                print(f"Ошибка: Строка {i+1} имеет длину {len(row)}, ожидалось {m}.", file=sys.stderr)
                validation_error = True
                break
            if not all(c in '#.' for c in row):
                print(f"Ошибка: Строка {i+1} содержит недопустимые символы (разрешены только '#' и '.').", file=sys.stderr)
                validation_error = True
                break
    if validation_error:
        try:
            open('OUTPUT.TXT', 'w').close()
        except:
            pass
        return

    # --- 2. Инициализация ---
    # Матрица для отслеживания посещенных клеток '#'
    visited = [[False for _ in range(m)] for _ in range(n)]
    # Счетчик связных компонент (кусков бумаги)
    component_count = 0

    # --- 3. Реализация DFS ---
    def is_valid(r: int, c: int) -> bool:
        """
        Проверяет, находятся ли координаты (r, c) в пределах сетки.

        Args:
        ---------------
            r: Координата строки
            c: Координата столбца

        Returns:
        ---------------
            True, если координаты в пределах сетки, иначе False
        """
        return 0 <= r < n and 0 <= c < m

    # Используем стек для итеративного DFS, чтобы избежать проблем с лимитом рекурсии
    # Это более надежный подход для соревновательного программирования
    def dfs_iterative(start_r: int, start_c: int) -> None:
        """
        Итеративная функция поиска в глубину с использованием стека.

        Args:
        ---------------
            start_r: Начальная координата строки
            start_c: Начальная координата столбца
        """
        if not is_valid(start_r, start_c) or visited[start_r][start_c] or grid[start_r][start_c] == '.':
            # Не начинаем обход, если стартовая точка невалидна
            return

        stack = [(start_r, start_c)]
        # Помечаем стартовую точку
        visited[start_r][start_c] = True

        while stack:
            r, c = stack.pop()

            # Проверяем 4-х соседей
            for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:  # Вправо, Влево, Вниз, Вверх
                nr, nc = r + dr, c + dc

                # Если сосед валиден, является '#' и еще не посещен
                if is_valid(nr, nc) and not visited[nr][nc] and grid[nr][nc] == '#':
                    visited[nr][nc] = True
                    stack.append((nr, nc))  # Добавляем в стек для дальнейшего обхода

    # --- 4. Основной цикл обхода сетки ---
    # Итерируем по каждой клетке сетки
    for i in range(n):
        for j in range(m):
            # Если находим клетку бумаги ('#'), которая еще не была посещена,
            # значит, мы обнаружили начало новой связной компоненты.
            if grid[i][j] == '#' and not visited[i][j]:
                # Увеличиваем счетчик компонент
                component_count += 1
                # Запускаем итеративный DFS из этой клетки, чтобы найти и пометить
                # все клетки, принадлежащие этой компоненте.
                dfs_iterative(i, j)

    # --- 5. Вывод результата ---
    try:
        with open('OUTPUT.TXT', 'w', encoding='utf-8') as f:
            f.write(str(component_count))
    except IOError:
        print("Ошибка: Не удалось записать результат в файл OUTPUT.TXT.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при записи в файл OUTPUT.TXT: {e}", file=sys.stderr)

# Запуск основной функции решения
if __name__ == "__main__":
    solve()

### ВАША интуиция

Представьте себе лист бумаги как карту, где символы `#` обозначают сушу, а `.` - воду. Наша цель - посчитать, сколько отдельных островов суши есть на этой карте. Два участка суши считаются частью одного острова, если они соприкасаются сторонами (не по диагонали). Задача сводится к классической проблеме поиска связных компонент в графе, где каждая клетка `#` - это узел, а ребра соединяют соседние по горизонтали или вертикали узлы `#`.

Чтобы посчитать количество островов, мы можем применить систематический подход. Мы будем "сканировать" карту клетка за клеткой. Как только мы наткнемся на клетку суши (`#`), которую мы еще не посещали (то есть она не принадлежит ни одному из уже обнаруженных островов), мы понимаем, что нашли новый остров. В этот момент мы увеличиваем наш счетчик островов на единицу. Но теперь нам нужно убедиться, что мы не посчитаем другие части этого же острова как новые острова при дальнейшем сканировании.

Для этого, найдя первую клетку нового острова, мы запускаем процедуру "исследования" этого острова. Эта процедура, обычно реализуемая через поиск в глубину (DFS) или поиск в ширину (BFS), систематически обходит все клетки `#`, достижимые из стартовой точки через соседние клетки `#`. Во время этого обхода мы "помечаем" каждую посещенную клетку острова (например, используя вспомогательную матрицу `visited`), чтобы знать, что она уже учтена. Как только весь остров исследован и помечен, мы возвращаемся к нашему сканированию карты и продолжаем искать следующую непомеченную клетку `#`. Этот процесс гарантирует, что каждый остров будет обнаружен и посчитан ровно один раз.

### ВАШ скрин ОКнутой задачи

## Задача: Укладка плитки ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=50&id_problem=641)) [1 балл]

Бригаде строителей поручили уложить квадратной плиткой пол в виде шахматного узора на кухне. Однако при укладке возникли недочёты: в некоторых местах плитки одного цвета оказались расположены рядом (имеют общую сторону). По заданной схеме замощения нужно определить, какое минимальное число строителей могло участвовать в укладке плитки.

**Идея решения:**  
Предполагается, что каждый строитель укладывал плитку по своему участку, не взаимодействуя с другими. Ошибки в узоре (соседние плитки одного цвета) могут возникать, если работы выполнялись несколькими строителями с различными шаблонами укладки. Задача сводится к разбиению плоскости на части, где в каждом участке плитки выложены единообразно (без нарушений шахматного чередования), и минимальное число таких участков соответствует минимальному числу строителей.

---

## Входные данные

Входной файл `INPUT.TXT` содержит 8 строк, каждая из которых состоит из 8 символов `W` и `B`:
- Символ `W` обозначает белую плитку.
- Символ `B` обозначает чёрную плитку.

---

## Выходные данные

Выведите одно число — минимальное число строителей, которые могли участвовать в укладке плитки, чтобы получить такое замощение.

---

## Пример

**Входные данные:**
```
WBWBWBBW
BWBBWBWB
WBWWBWBW
WBWWBWWB
BWBBWBWB
WBWBWWBW
BWBWBBWB
WBWBWWBW
```

**Выходные данные:**
```
4
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
# -*- coding: utf-8 -*-
import sys
from typing import List, Tuple

# Константы для размера сетки
N_ROWS = 8
N_COLS = 8

def solve() -> None:
    """
    Основная функция для решения задачи о минимальном количестве строителей.
    Считает количество связных компонент на сетке 8x8, где связность
    определяется правильным шахматным чередованием цветов ('W' и 'B')
    соседних плиток. Использует итеративный поиск в глубину (DFS).
    """
    # --- 1. Чтение входных данных ---
    grid: List[str] = []
    try:
        with open('INPUT.TXT', 'r', encoding='utf-8') as f:
            for i in range(N_ROWS):
                line = f.readline()
                # Проверка на недостаток строк
                if not line:
                    print(f"Ошибка: Неожиданный конец файла. Ожидалось {N_ROWS} строк, прочитано {i}.", file=sys.stderr)
                    try:
                        open('OUTPUT.TXT', 'w').close()  # Создать пустой файл при ошибке
                    except:
                        pass
                    return
                line = line.strip()
                # Проверка длины строки
                if len(line) != N_COLS:
                    print(f"Ошибка: Строка {i+1} имеет длину {len(line)}, ожидалось {N_COLS}.", file=sys.stderr)
                    try:
                        open('OUTPUT.TXT', 'w').close()
                    except:
                        pass
                    return
                # Проверка допустимых символов
                valid_chars = 'WB'
                if not all(c in valid_chars for c in line):
                    invalid_chars = ''.join(sorted(list(set(c for c in line if c not in valid_chars))))
                    print(f"Ошибка: Строка {i+1} содержит недопустимые символы '{invalid_chars}' (разрешены только 'W' и 'B').", file=sys.stderr)
                    try:
                        open('OUTPUT.TXT', 'w').close()
                    except:
                        pass
                    return
                grid.append(line)
        # Проверка, что прочитано ровно N_ROWS строк (на случай лишних строк)
        if len(grid) != N_ROWS:
            print(f"Ошибка: Прочитано {len(grid)} строк, ожидалось {N_ROWS}.", file=sys.stderr)
            try:
                open('OUTPUT.TXT', 'w').close()
            except:
                pass
            return

    except FileNotFoundError:
        print("Ошибка: Файл INPUT.TXT не найден.", file=sys.stderr)
        try:
            open('OUTPUT.TXT', 'w').close()
        except:
            pass
        return
    except Exception as e:
        print(f"Ошибка при чтении файла INPUT.TXT: {e}", file=sys.stderr)
        try:
            open('OUTPUT.TXT', 'w').close()
        except:
            pass
        return

    # --- 2. Инициализация ---
    visited: List[List[bool]] = [[False for _ in range(N_COLS)] for _ in range(N_ROWS)]
    component_count = 0

    # --- 3. Реализация итеративного DFS ---
    def is_valid(r: int, c: int) -> bool:
        """
        Проверяет, находятся ли координаты (r, c) в пределах сетки.

        Args:
        ---------------
            r: Координата строки
            c: Координата столбца

        Returns:
        ---------------
            True, если координаты в пределах сетки, иначе False
        """
        return 0 <= r < N_ROWS and 0 <= c < N_COLS

    def dfs_iterative(start_r: int, start_c: int) -> None:
        """
        Итеративный DFS для обхода одной связной компоненты.

        Args:
        ---------------
            start_r: Начальная координата строки
            start_c: Начальная координата столбца
        """
        if visited[start_r][start_c]:  # Доп. проверка, хотя не должна срабатывать
            return

        stack: List[Tuple[int, int]] = [(start_r, start_c)]
        visited[start_r][start_c] = True

        while stack:
            curr_r, curr_c = stack.pop()
            current_color = grid[curr_r][curr_c]

            # Направления смещения к соседям
            directions: List[Tuple[int, int]] = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Право, Лево, Низ, Верх

            for dr, dc in directions:
                nr, nc = curr_r + dr, curr_c + dc

                # Условия для добавления соседа в стек:
                # 1. В пределах сетки
                # 2. Не посещен
                # 3. Цвет отличается (правило шахматной доски)
                if is_valid(nr, nc) and \
                   not visited[nr][nc] and \
                   grid[nr][nc] != current_color:
                    visited[nr][nc] = True
                    stack.append((nr, nc))

    # --- 4. Основной цикл обхода сетки ---
    for i in range(N_ROWS):
        for j in range(N_COLS):
            # Если клетка не посещена, начинаем обход новой компоненты
            if not visited[i][j]:
                component_count += 1
                dfs_iterative(i, j)

    # --- 5. Вывод результата ---
    try:
        with open('OUTPUT.TXT', 'w', encoding='utf-8') as f:
            f.write(str(component_count))
    except IOError:
        print("Ошибка: Не удалось записать результат в файл OUTPUT.TXT.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при записи в файл OUTPUT.TXT: {e}", file=sys.stderr)

# Запуск основной функции решения
if __name__ == "__main__":
    solve()

### ВАША интуиция

Задача сводится к разбиению сетки 8×8 на минимальное количество связных областей, где в каждой области соблюдается правило шахматного узора: соседние плитки имеют разные цвета. Каждая такая область соответствует участку, который мог выложить один строитель. Нарушения узора (соседние плитки одного цвета) возникают на границах между областями, и наша цель — определить минимальное число таких областей, чтобы объяснить все нарушения.

Для решения используется поиск в глубину (DFS). Алгоритм последовательно обходит все клетки сетки, начиная с непосещённых. При этом для каждой клетки проверяются её соседи: если соседняя клетка имеет противоположный цвет и ещё не посещена, она добавляется в текущую область. Таким образом, каждая итерация DFS находит одну связную область с корректным шахматным узором. Процесс повторяется до полного обхода сетки, и количество запусков DFS равно числу областей.

### ВАШ скрин ОКнутой задачи

## Задача: Раздел империи ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=50&id_problem=1217)) [1 балл]

Великая империя прошлого состояла из N городов, соединённых M двусторонними дорогами таким образом, что между любыми двумя городами существовал путь (возможно, через другие города). При этом:
- Между одной и той же парой городов может проходить несколько дорог.
- Дороги могут выходить и входить в один и тот же город.

Со временем K городов усилились и возвысились над остальными, между ними начались междоусобицы, и в результате империя распалась на K государств, в каждом из которых столицей стал один из усиленных городов.

Из древних источников неизвестно, какой город каким государством оказался после распада империи. Известно, однако, что в каждом из образованных государств, как и в великой империи, между любой парой городов существует путь (через другие города того же государства), и каждый город империи вошёл ровно в одно государство.

**Задача:** Восстановить карту, то есть определить разбиение городов империи по государствам. Для каждого усиленного города (столицы) необходимо вывести список городов, принадлежащих соответствующему государству. Если вариантов решения несколько, можно вывести любой.

---

## Входные данные

- Первая строка входного файла `INPUT.TXT` содержит два целых числа:
  - N (1 ≤ N ≤ 1000) — количество городов;
  - M (N-1 ≤ M ≤ 10^5) — количество дорог.
- Следующие M строк содержат описание дорог. Каждая строка состоит из двух целых чисел x_i и y_i (1 ≤ x_i, y_i ≤ N), означающих, что дорога соединяет города с номерами x_i и y_i.
- Следующая строка содержит целое число K (1 ≤ K ≤ N) — количество усиленных городов (столиц).
- Последняя строка содержит K различных целых чисел c_i (1 ≤ c_i ≤ N) — номера городов, ставших столицами.

---

## Выходные данные

В выходном файле `OUTPUT.TXT` для каждого из K усиленных городов необходимо вывести следующее:
- В первой строке — количество городов в государстве, где столицей является этот город.
- Во второй строке — номера городов, входящих в это государство.

Порядок вывода для усиленных городов должен соответствовать порядку их перечисления во входном файле.

---

## Пример

**Входные данные:**
```
3 2
1 2
2 3
2
1 3
```

**Выходные данные:**
```
2
1 2
1
3
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
# -*- coding: utf-8 -*-
import sys
from collections import deque
from typing import List, Dict, Tuple

# Имена файлов ввода/вывода
input_filename = "INPUT.TXT"
output_filename = "OUTPUT.TXT"

def solve() -> None:
    """
    Решает задачу восстановления карты империи после распада.
    Использует BFS из нескольких источников (столиц) для определения
    принадлежности городов к государствам. Читает из INPUT.TXT, пишет в OUTPUT.TXT.
    """
    capitals_input_order: List[int] = []  # Сохраним исходный порядок столиц для вывода
    capitals: List[int] = []  # Список валидных столиц
    n = 0
    m = 0
    k = 0
    adj: List[List[int]] = []

    try:
        # Чтение входных данных из файла
        with open(input_filename, 'r') as f_in:
            # Чтение N, M и базовая валидация
            line1 = f_in.readline()
            if not line1:
                raise EOFError("Файл пуст или не содержит первой строки (N, M)")
            n_str, m_str = line1.split()
            n = int(n_str)
            m = int(m_str)
            if not (1 <= n <= 1000):
                raise ValueError(f"N={n} вне [1, 1000]")
            if not (0 <= m <= 10**5):
                raise ValueError(f"M={m} вне [0, 10^5]")

            adj = [[] for _ in range(n + 1)]
            for i in range(m):
                line = f_in.readline()
                if not line:
                    raise EOFError(f"Ожидалось ребро {i+1}/{m}, получен EOF")
                u_str, v_str = line.split()
                u = int(u_str)
                v = int(v_str)
                # Валидация ребер
                if not (1 <= u <= n and 1 <= v <= n):
                    continue  # Игнорируем некорректное ребро
                adj[u].append(v)
                adj[v].append(u)

            # Чтение K и валидация
            line_k = f_in.readline()
            if not line_k:
                raise EOFError("Ожидалось K, получен EOF")
            k = int(line_k)
            if not (1 <= k <= n):
                raise ValueError(f"K={k} вне [1, N={n}]")

            # Чтение столиц и валидация
            line_capitals = f_in.readline()
            if not line_capitals:
                raise EOFError("Ожидался список столиц, получен EOF")
            capitals_input_order = list(map(int, line_capitals.split()))

            if len(capitals_input_order) != k:
                raise ValueError(f"Ожидалось K={k} столиц, получено {len(capitals_input_order)}")

            seen_capitals = set()
            for c in capitals_input_order:
                if not (1 <= c <= n):
                    raise ValueError(f"Некорректный номер столицы: {c}")
                if c in seen_capitals:
                    raise ValueError(f"Столица {c} указана более одного раза")
                capitals.append(c)  # Добавляем только валидные и уникальные
                seen_capitals.add(c)

    except FileNotFoundError:
        # Создаем пустой файл при ошибке
        with open(output_filename, "w") as f_out:
            pass
        return
    except (ValueError, EOFError, IndexError) as e:
        # Создаем пустой файл при ошибке
        with open(output_filename, "w") as f_out:
            pass
        return
    except Exception as e:
        # Создаем пустой файл при ошибке
        with open(output_filename, "w") as f_out:
            pass
        return

    # --- Основная логика BFS ---
    owner: List[int] = [0] * (n + 1)  # owner[i] = c, если город i принадлежит столице c
    q: deque = deque()

    # Начальное состояние BFS: добавляем все (валидные) столицы
    for capital in capitals:
        owner[capital] = capital
        q.append(capital)

    # Запускаем BFS
    processed_nodes_count = 0
    while q:
        u = q.popleft()
        processed_nodes_count += 1

        for v in adj[u]:
            if 1 <= v <= n and owner[v] == 0:
                owner[v] = owner[u]  # Присваиваем ту же столицу
                q.append(v)

    # Формирование результата
    states: Dict[int, List[int]] = {c: [] for c in capitals}
    for i in range(1, n + 1):
        if owner[i] != 0:  # Если город был достигнут
            states[owner[i]].append(i)

    # Запись результата в файл
    try:
        with open(output_filename, "w") as f_out:
            output_lines: List[str] = []
            for capital in capitals_input_order:
                if capital in states:
                    result_list = states[capital]
                    output_lines.append(str(len(result_list)))
                    output_lines.append(" ".join(map(str, sorted(result_list))))
                else:
                    output_lines.append("0")
                    output_lines.append("")

            f_out.write("\n".join(output_lines) + "\n")
    except IOError:
        pass
    except Exception:
        pass

# Вызов основной функции решения
solve()

### ВАША интуиция
Задача сводится к разбиению графа на K непересекающихся подграфов, где каждый подграф содержит ровно одну из заданных столиц. Важно отметить, что между любыми двумя городами в исходном графе существует путь, то есть граф связный. Разбиение должно быть таким, чтобы каждый город принадлежал ровно одному подграфу, и внутри каждого подграфа сохранялась связность.

Для решения используется обход графа в ширину (BFS) из нескольких источников — заданных столиц. Каждая столица "захватывает" все города, которые могут быть достигнуты из неё раньше, чем из других столиц. Это гарантирует, что каждый город будет отнесён к ближайшей столице, а значит, разбиение будет корректным. Если несколько столиц находятся на одинаковом расстоянии от города, выбор одной из них произвольный, так как задача допускает любое решение.

### ВАШ скрин ОКнутой задачи

## Задача: Табличка ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=51&id_problem=650)) [1 балл]
Вам дана таблица, состоящая из N строк и M столбцов. В каждой клетке таблицы содержится число 0 или 1. Расстояние между клетками с координатами (x₁, y₁) и (x₂, y₂) определяется как |x₁ - x₂| + |y₁ - y₂|.

**Задача:** Построить новую таблицу того же размера, в которой для каждой клетки будет указано расстояние до ближайшей клетки, содержащей 1 (из исходной таблицы). Гарантируется, что в исходной таблице есть хотя бы одна клетка с единицей.

---

## Входные данные

- Первая строка входного файла `INPUT.TXT` содержит два натуральных числа N и M (N, M ≤ 100) — количество строк и столбцов таблицы.
- Далее следуют N строк, каждая содержащая M чисел (0 или 1) — элементы таблицы.

---

## Выходные данные

Выведите в выходной файл `OUTPUT.TXT` N строк, каждая из которых содержит M чисел — элементы искомой таблицы, где каждое число является расстоянием от соответствующей клетки исходной таблицы до ближайшей клетки с единицей.

---

## Пример

**Входные данные:**
```
2 3
0 0 1
1 0 0
```

**Выходные данные:**
```
1 1 0
0 1 1
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
import collections
from typing import List, Tuple

INFINITY = float('inf')

def read_input(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Description:
    ---------------
        Считывает входные данные из файла.

    Args:
    ---------------
        file_path: Путь к файлу с входными данными

    Returns:
    ---------------
        Кортеж с количеством строк, столбцов и списком значений сетки

    Raises:
    ---------------
        Exception: В случае ошибки чтения файла

    Examples:
    ---------------
        >>> read_input("INPUT.TXT")
        (3, 3, [[0, 1, 0], [0, 0, 0], [0, 0, 1]])
    """
    try:
        with open(file_path, "r") as file:
            n, m = map(int, file.readline().split())
            grid = [list(map(int, file.readline().split())) for _ in range(n)]
        return n, m, grid
    except Exception as e:
        print(f"Критическая ошибка при чтении INPUT.TXT: {e}")
        exit(1)

def initialize_distances(n: int, m: int) -> List[List[int]]:
    """
    Description:
    ---------------
        Инициализирует матрицу расстояний.

    Args:
    ---------------
        n: Количество строк
        m: Количество столбцов

    Returns:
    ---------------
        Матрица расстояний, заполненная бесконечностями

    Examples:
    ---------------
        >>> initialize_distances(3, 3)
        [[inf, inf, inf], [inf, inf, inf], [inf, inf, inf]]
    """
    return [[INFINITY] * m for _ in range(n)]

def find_sources(grid: List[List[int]], distances: List[List[int]]) -> collections.deque:
    """
    Description:
    ---------------
        Находит источники (клетки с '1') и инициализирует их расстояния.

    Args:
    ---------------
        grid: Сетка значений
        distances: Матрица расстояний

    Returns:
    ---------------
        Очередь с координатами источников

    Examples:
    ---------------
        >>> find_sources([[0, 1, 0], [0, 0, 0], [0, 0, 1]], initialize_distances(3, 3))
        deque([(0, 1), (2, 2)])
    """
    q = collections.deque()
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == 1:
                distances[r][c] = 0
                q.append((r, c))
    return q

def bfs(grid: List[List[int]], distances: List[List[int]], q: collections.deque) -> None:
    """
    Description:
    ---------------
        Выполняет поиск в ширину (BFS) для вычисления кратчайших расстояний.

    Args:
    ---------------
        grid: Сетка значений
        distances: Матрица расстояний
        q: Очередь с координатами источников

    Examples:
    ---------------
        >>> distances = initialize_distances(3, 3)
        >>> q = find_sources([[0, 1, 0], [0, 0, 0], [0, 0, 1]], distances)
        >>> bfs([[0, 1, 0], [0, 0, 0], [0, 0, 1]], distances, q)
        >>> distances
        [[2, 0, 1], [1, 2, 3], [0, 1, 0]]
    """
    # Определяем смещения для соседей (вверх, вниз, влево, вправо)
    dr = [-1, 1, 0, 0]
    dc = [0, 0, -1, 1]

    while q:
        r, c = q.popleft()
        current_dist = distances[r][c]

        # Перебираем 4 возможных направления
        for i in range(4):
            nr, nc = r + dr[i], c + dc[i]

            # Проверяем, что сосед находится в пределах таблицы
            if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
                # Если через текущую клетку (r, c) путь до соседа (nr, nc) короче
                # и мы еще не нашли оптимальный путь до него
                if distances[nr][nc] > current_dist + 1:
                    distances[nr][nc] = current_dist + 1
                    q.append((nr, nc))  # Добавляем соседа в очередь для дальнейшей обработки

def write_output(file_path: str, distances: List[List[int]]) -> None:
    """
    Description:
    ---------------
        Записывает результат в файл.

    Args:
    ---------------
        file_path: Путь к файлу для записи результата
        distances: Матрица расстояний

    Raises:
    ---------------
        Exception: В случае ошибки записи файла

    Examples:
    ---------------
        >>> write_output("OUTPUT.TXT", [[2, 0, 1], [1, 2, 3], [0, 1, 0]])
    """
    try:
        with open(file_path, "w") as file:
            for row in distances:
                file.write(" ".join(map(str, row)) + "\n")
    except Exception as e:
        print(f"Критическая ошибка при записи OUTPUT.TXT: {e}")
        exit(1)

def main():
    """
    Description:
    ---------------
        Основная функция для выполнения алгоритма.
    """
    n, m, grid = read_input("INPUT.TXT")
    distances = initialize_distances(n, m)
    q = find_sources(grid, distances)
    bfs(grid, distances, q)
    write_output("OUTPUT.TXT", distances)

if __name__ == "__main__":
    main()

### ВАША интуиция

Задача заключается в том, чтобы для каждой клетки таблицы определить расстояние до ближайшей клетки, содержащей 1. Это можно представить как задачу поиска кратчайших путей от всех клеток с 1 до остальных клеток таблицы. Чтобы решить её эффективно, мы используем алгоритм поиска в ширину (BFS), который идеально подходит для работы с невзвешенными графами. Этот подход гарантирует, что каждая клетка будет обработана по кратчайшему пути до ближайшей единицы.

### ВАШ скрин ОКнутой задачи

## Задача: Звезда ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=51&id_problem=652)) [0.5 балла]

Вам дана карта звёздного неба, представленная прямоугольной таблицей из N строк и M столбцов. Каждая ячейка таблицы может содержать один из двух символов:
- Точка (.) — означает отсутствие звезды.
- Звездочка (*) — означает наличие звезды.

**Созвездие** — это группа звёзд, которые соединены между собой по горизонтали или вертикали (то есть звёзды считаются соединёнными, если они являются соседями сверху, снизу, справа или слева). При этом одна изолированная звезда также считается созвездием.

**Задача:** Написать программу, которая подсчитывает количество созвездий на карте.

---

## Входные данные

- Первая строка входного файла `INPUT.TXT` содержит два натуральных числа N и M (N, M ≤ 300) — количество строк и столбцов карты.
- Далее следуют N строк, каждая из которых содержит M символов: «.» (точка) или «*» (звездочка).

---

## Выходные данные

Выведите одно число — количество созвездий на карте.

---

## Пример

**Входные данные:**
```
8 12
..**........
*..*..***...
*..*...*..*.
*****..*....
.......*...*
.......*..*.
....*.......
..********..
```

**Выходные данные:**
```
6
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
import sys
from typing import List, Tuple

def read_input(file_path: str) -> Tuple[int, int, List[List[str]]]:
    """
    Description:
    ---------------
        Считывает входные данные из файла.

    Args:
    ---------------
        file_path: Путь к файлу с входными данными

    Returns:
    ---------------
        Кортеж с количеством строк, столбцов и списком значений сетки

    Raises:
    ---------------
        Exception: В случае ошибки чтения файла или невалидного формата данных

    Examples:
    ---------------
        >>> read_input("INPUT.TXT")
        (3, 3, [['.', '*', '.'], ['.', '.', '.'], ['*', '.', '*']])
    """
    try:
        with open(file_path, "r") as file:
            line1 = file.readline().split()
            if len(line1) != 2:
                # Невалидный формат первой строки
                exit(1)
            n_str, m_str = line1
            # Проверка, что N и M действительно числа
            if not n_str.isdigit() or not m_str.isdigit():
                # N или M не являются числами
                exit(1)
            n, m = int(n_str), int(m_str)

            # Проверка ограничений N и M
            if not (1 <= n <= 300 and 1 <= m <= 300):
                # N или M вне допустимого диапазона
                exit(1)

            grid = [list(file.readline().strip()) for _ in range(n)]

            # Проверка фактических размеров прочитанной карты
            if len(grid) != n or any(len(row) != m for row in grid):
                # Несоответствие размеров
                exit(1)

            # Проверка допустимых символов
            allowed_chars = {'.', '*'}
            for r in range(n):
                for c in range(m):
                    if grid[r][c] not in allowed_chars:
                        # Недопустимый символ в карте
                        exit(1)

        return n, m, grid
    except FileNotFoundError:
        # Файл не найден
        exit(1)
    except Exception as e:
        print(f"Error during input processing: {e}", file=sys.stderr)
        exit(1)

def initialize_visited(n: int, m: int) -> List[List[bool]]:
    """
    Description:
    ---------------
        Инициализирует матрицу посещенных клеток.

    Args:
    ---------------
        n: Количество строк
        m: Количество столбцов

    Returns:
    ---------------
        Матрица посещенных клеток, заполненная False

    Examples:
    ---------------
        >>> initialize_visited(3, 3)
        [[False, False, False], [False, False, False], [False, False, False]]
    """
    return [[False] * m for _ in range(n)]

def mark_component(grid: List[List[str]], visited: List[List[bool]], start_r: int, start_c: int) -> None:
    """
    Description:
    ---------------
        Итеративно обходит и маркирует созвездие.

    Args:
    ---------------
        grid: Сетка значений
        visited: Матрица посещенных клеток
        start_r: Начальная строка
        start_c: Начальный столбец

    Examples:
    ---------------
        >>> visited = initialize_visited(3, 3)
        >>> mark_component([['.', '*', '.'], ['.', '.', '.'], ['*', '.', '*']], visited, 0, 1)
        >>> visited
        [[False, True, False], [False, False, False], [True, False, True]]
    """
    # Определяем смещения для соседей (вверх, вниз, влево, вправо)
    dr = [-1, 1, 0, 0]
    dc = [0, 0, -1, 1]

    # Используем стек для итеративного DFS
    stack = [(start_r, start_c)]
    visited[start_r][start_c] = True  # Помечаем стартовую клетку

    while stack:
        curr_r, curr_c = stack.pop()

        # Обходим соседей
        for i in range(4):
            nr, nc = curr_r + dr[i], curr_c + dc[i]

            # Проверка границ, типа клетки ('*') и посещенности
            if (0 <= nr < len(grid) and 0 <= nc < len(grid[0])
                    and not visited[nr][nc]
                    and grid[nr][nc] == '*'):
                visited[nr][nc] = True  # Помечаем как посещенную
                stack.append((nr, nc))  # Добавляем в стек для обхода

def write_output(file_path: str, constellation_count: int) -> None:
    """
    Description:
    ---------------
        Записывает результат в файл.

    Args:
    ---------------
        file_path: Путь к файлу для записи результата
        constellation_count: Количество созвездий

    Raises:
    ---------------
        Exception: В случае ошибки записи файла

    Examples:
    ---------------
        >>> write_output("OUTPUT.TXT", 2)
    """
    try:
        with open(file_path, "w") as file:
            file.write(str(constellation_count) + "\n")
    except Exception as e:
        # Ошибка записи файла
        print(f"Error writing output file: {e}", file=sys.stderr)
        exit(1)

def solve():
    """
    Description:
    ---------------
        Основная функция для выполнения алгоритма.
    """
    n, m, grid = read_input("INPUT.TXT")
    visited = initialize_visited(n, m)
    constellation_count = 0

    # Основной цикл обхода карты
    for r in range(n):
        for c in range(m):
            # Если нашли непосещенную звезду - это новое созвездие
            if grid[r][c] == '*' and not visited[r][c]:
                constellation_count += 1
                mark_component(grid, visited, r, c)  # Запускаем обход для пометки всей компоненты

    write_output("OUTPUT.TXT", constellation_count)

if __name__ == "__main__":
    solve()

### ВАША интуиция

Задача заключается в подсчёте групп звёзд на карте, где звёзды считаются связанными, если они расположены рядом по вертикали или горизонтали. Каждая такая группа, даже состоящая из одной звезды, образует созвездие. Чтобы найти все созвездия, нужно проверить каждую клетку карты и, наткнувшись на неотмеченную звезду, найти все связанные с ней звёзды, двигаясь вверх, вниз, влево и вправо.

Алгоритм работает так: создается таблица для отметки уже проверенных звёзд. Затем карта просматривается по клеткам. Если встречается звезда, которую ещё не проверяли, это новое созвездие. Из этой звезды начинается поиск всех соседних звёзд (например, с помощью стека, как в коде), и все найденные помечаются, чтобы не учитывать их повторно. Так повторяется, пока все звёзды не будут обработаны.

Подход гарантирует, что каждая звезда будет проверена ровно один раз. Это эффективно: время работы пропорционально размеру карты (N×M), а память используется для хранения таблицы посещений. Благодаря этому даже для максимальных размеров карты (300×300) решение работает быстро.

### ВАШ скрин ОКнутой задачи

## Задача: Морской бой-3 ([ссылка](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=21&id_topic=51&id_problem=653)) [0.5 балла]

Всем известна игра «Морской бой». В данной задаче поле имеет произвольные размеры N×M (1 ≤ N, M ≤ 10³). На поле расположены корабли, которые могут иметь любые формы и размеры. Корабль определяется как связная фигура клеток, соединённых по сторонам.

Каждая клетка игрового поля может содержать один из следующих символов:
- 'X' – подбитая клетка корабля;
- 'S' – не подбитая клетка корабля;
- '-' – свободная водная клетка.

**Задача:** Необходимо вывести три числа, разделённых пробелом:
1. Количество **целых кораблей** – корабли, у которых ни одна клетка не повреждена (все клетки обозначены символом 'S').
2. Количество **подбитых кораблей** – корабли, у которых имеются и подбитые ('X'), и неповреждённые ('S') клетки.
3. Количество **уничтоженных кораблей** – корабли, у которых все клетки подбиты (только символы 'X').

Корабли могут иметь любую форму, но клетки корабля соединены по сторонам.

---

## Входные данные

- Первая строка входного файла `INPUT.TXT` содержит два целых числа N и M (1 ≤ N, M ≤ 10³) – размеры игрового поля (N – число строк, M – число столбцов).
- Далее следуют N строк, каждая из которых содержит M символов – описание игрового поля.

---

## Выходные данные

Выведите в выходной файл `OUTPUT.TXT` через пробел три числа:
- количество целых кораблей,
- количество подбитых кораблей,
- количество уничтоженных кораблей.

---

## Пример

**Входные данные:**
```
3 8
---SSS--
XX--S-X-
X-S---S-
```

**Выходные данные:**
```
2 1 1
```

In [ ]:
# Асимптотическая сложность по времени - О(TODO)
# Асимптотическая сложность по памяти - O(TODO)

# ВАШ КОД
# Не получается решить, постоянно, то Memory limit exceeded, то Time limit exceeded, сложновата задач...)

### ВАША интуиция

### ВАШ скрин ОКнутой задачи